### 목표 : 당뇨병 예측 모델 구현

-   데이터 : scikit-learn dataset
-   피쳐 :
-   타겟 :


(1) 모듈 로딩 및 데이터 준비


In [1]:
from sklearnex import patch_sklearn

patch_sklearn()
from sklearn.datasets import load_diabetes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import (
    StandardScaler,
    MinMaxScaler,
    RobustScaler,
    QuantileTransformer,
)
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy import stats


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
data_file = "./diabetes.tab.txt"


In [3]:
diabetesDF = pd.read_csv(data_file, sep="\t", header=0)
diabetesDF.columns = ["AGE", "SEX", "BMI", "BP", "tc", "ldl", "hdl", "tch", "ltg", "glu", "Y",]  # fmt: skip
diabetesDF


,AGE,SEX,BMI,BP,tc,ldl,hdl,tch,ltg,glu,Y
0,59,2,32.1,101.00,157,93.2,38.0,4.00,4.8598,87,151
1,48,1,21.6,87.00,183,103.2,70.0,3.00,3.8918,69,75
2,72,2,30.5,93.00,156,93.6,41.0,4.00,4.6728,85,141
3,24,1,25.3,84.00,198,131.4,40.0,5.00,4.8903,89,206
4,50,1,23.0,101.00,192,125.4,52.0,4.00,4.2905,80,135
...,...,...,...,...,...,...,...,...,...,...,...
437,60,2,28.2,112.00,185,113.8,42.0,4.00,4.9836,93,178
438,47,2,24.9,75.00,225,166.0,42.0,5.00,4.4427,102,104
439,60,2,24.9,99.67,162,106.6,43.0,3.77,4.1271,95,132
440,36,1,30.0,95.00,201,125.2,42.0,4.79,5.1299,85,220


In [4]:
z = np.abs(stats.zscore(diabetesDF))

threshold = 3
print(np.where(z > threshold))
diabetesDF = diabetesDF[(z < threshold).all(axis=1)]
diabetesDF.reset_index(drop=True, inplace=True)
diabetesDF


(array([ 58, 123, 123, 123, 216, 230, 230, 256, 260, 261, 269, 322, 336,
       367, 441], dtype=int64), array([6, 4, 5, 7, 7, 4, 5, 2, 6, 6, 6, 7, 7, 2, 6], dtype=int64))


,AGE,SEX,BMI,BP,tc,ldl,hdl,tch,ltg,glu,Y
0,59,2,32.1,101.00,157,93.2,38.0,4.00,4.8598,87,151
1,48,1,21.6,87.00,183,103.2,70.0,3.00,3.8918,69,75
2,72,2,30.5,93.00,156,93.6,41.0,4.00,4.6728,85,141
3,24,1,25.3,84.00,198,131.4,40.0,5.00,4.8903,89,206
4,50,1,23.0,101.00,192,125.4,52.0,4.00,4.2905,80,135
...,...,...,...,...,...,...,...,...,...,...,...
425,33,1,19.5,80.00,171,85.4,75.0,2.00,3.9703,80,48
426,60,2,28.2,112.00,185,113.8,42.0,4.00,4.9836,93,178
427,47,2,24.9,75.00,225,166.0,42.0,5.00,4.4427,102,104
428,60,2,24.9,99.67,162,106.6,43.0,3.77,4.1271,95,132


In [5]:
diabetesDF.describe()


,AGE,SEX,BMI,BP,tc,ldl,hdl,tch,ltg,glu,Y
count,430.000000,430.000000,430.000000,430.000000,430.000000,430.000000,430.000000,430.000000,430.000000,430.000000,430.000000
mean,48.504651,1.469767,26.289302,94.674372,188.025581,114.847209,49.477907,4.043395,4.634329,91.102326,151.393023
std,13.203578,0.499667,4.281714,13.767554,33.803033,29.473002,11.969857,1.218012,0.519470,11.491712,76.278513
min,19.000000,1.000000,18.000000,62.000000,97.000000,41.600000,22.000000,2.000000,3.258100,58.000000,25.000000
25%,38.250000,1.000000,23.200000,85.000000,164.000000,96.050000,41.000000,3.000000,4.266200,83.000000,87.250000
50%,50.000000,1.000000,25.700000,93.000000,186.000000,113.000000,48.000000,4.000000,4.605200,91.000000,140.000000
75%,59.000000,2.000000,29.200000,104.917500,208.000000,133.450000,57.000000,5.000000,4.988700,98.000000,207.500000
max,79.000000,2.000000,39.100000,133.000000,286.000000,199.400000,88.000000,7.900000,6.107000,124.000000,341.000000


In [6]:
diabetesDF.corr()


,AGE,SEX,BMI,BP,tc,ldl,hdl,tch,ltg,glu,Y
AGE,1.000000,0.178802,0.209488,0.328698,0.277281,0.243806,-0.101100,0.231392,0.283381,0.312350,0.203801
SEX,0.178802,1.000000,0.074700,0.240276,0.026613,0.127081,-0.370851,0.320979,0.140353,0.197022,0.037038
BMI,0.209488,0.074700,1.000000,0.427855,0.250807,0.250363,-0.346641,0.392450,0.440331,0.380474,0.574210
BP,0.328698,0.240276,0.427855,1.000000,0.267658,0.213246,-0.191702,0.276519,0.400635,0.400283,0.457891
tc,0.277281,0.026613,0.250807,0.267658,1.000000,0.900119,0.045392,0.544248,0.522386,0.310482,0.221637
ldl,0.243806,0.127081,0.250363,0.213246,0.900119,1.000000,-0.184463,0.656649,0.317755,0.270712,0.176021
hdl,-0.101100,-0.370851,-0.346641,-0.191702,0.045392,-0.184463,1.000000,-0.746193,-0.400346,-0.266172,-0.387524
tch,0.231392,0.320979,0.392450,0.276519,0.544248,0.656649,-0.746193,1.000000,0.615142,0.395053,0.424691
ltg,0.283381,0.140353,0.440331,0.400635,0.522386,0.317755,-0.400346,0.615142,1.000000,0.454386,0.558506
glu,0.312350,0.197022,0.380474,0.400283,0.310482,0.270712,-0.266172,0.395053,0.454386,1.000000,0.379788


In [7]:
featureDF = diabetesDF[["SEX", "BP", "BMI", "ltg"]]
# featureDF = diabetesDF[diabetesDF.columns[:-1]]
targetDF = diabetesDF["Y"]


In [8]:
scalers = []
scalers.append(StandardScaler())
scalers.append(MinMaxScaler())
scalers.append(RobustScaler())
scalers.append(QuantileTransformer())


In [9]:
scoreDict = {}
i = 1
for num in range(100):
    x_train, x_test, y_train, y_test = train_test_split(
        featureDF, targetDF, test_size=0.2, random_state=num
    )
    for scaler in scalers:
        scaler.fit(x_train)
        scaled_x_train = scaler.transform(x_train)
        scaled_x_test = scaler.transform(x_test)
        for weight in ["uniform", "distance"]:
            for algorithm in ["auto", "ball_tree", "kd_tree", "brute"]:
                # for k in range(1, scaled_x_train.shape[0]):
                for k in range(1, 50 + 1):
                    model = KNeighborsRegressor(
                        n_neighbors=k, weights=weight, algorithm=algorithm, n_jobs=-1
                    )
                    model.fit(scaled_x_train, y_train)
                    t1 = model.score(scaled_x_train, y_train)
                    t2 = model.score(scaled_x_test, y_test)
                    if t2 > 0.5:
                        scoreDict[i] = (
                            num,
                            scaler,
                            weight,
                            algorithm,
                            k,
                            t1,
                            t2,
                            abs(t2 - t1),
                        )
                        i += 1

num, scaler, weight, algorithm, k, t1, t2, diff = max(
    scoreDict.values(), key=lambda x: x[6]
)


c:\Users\KDP-25\.conda\envs\MachineLearning39\lib\site-packages\sklearn\preprocessing\_data.py:2667: UserWarning: n_quantiles (1000) is greater than the total number of samples (344). n_quantiles is set to n_samples.
  warnings.warn(
c:\Users\KDP-25\.conda\envs\MachineLearning39\lib\site-packages\sklearn\preprocessing\_data.py:2667: UserWarning: n_quantiles (1000) is greater than the total number of samples (344). n_quantiles is set to n_samples.
  warnings.warn(
c:\Users\KDP-25\.conda\envs\MachineLearning39\lib\site-packages\sklearn\preprocessing\_data.py:2667: UserWarning: n_quantiles (1000) is greater than the total number of samples (344). n_quantiles is set to n_samples.
  warnings.warn(
c:\Users\KDP-25\.conda\envs\MachineLearning39\lib\site-packages\sklearn\preprocessing\_data.py:2667: UserWarning: n_quantiles (1000) is greater than the total number of samples (344). n_quantiles is set to n_samples.
  warnings.warn(
c:\Users\KDP-25\.conda\envs\MachineLearning39\lib\site-packages\

In [10]:
scaler, k, t1, t2, diff


(RobustScaler(),
 10,
 0.490020473393589,
 0.6288849806298598,
 0.13886450723627075)

In [11]:
sorted(scoreDict.values(), key=lambda x: x[6], reverse=True)


[(61,
  RobustScaler(),
  'uniform',
  'auto',
  10,
  0.490020473393589,
  0.6288849806298598,
  0.13886450723627075),
 (61,
  RobustScaler(),
  'uniform',
  'ball_tree',
  10,
  0.490020473393589,
  0.6288849806298598,
  0.13886450723627075),
 (61,
  RobustScaler(),
  'uniform',
  'kd_tree',
  10,
  0.490020473393589,
  0.6288849806298598,
  0.13886450723627075),
 (61,
  StandardScaler(),
  'uniform',
  'auto',
  12,
  0.4793929703153893,
  0.6264927985225471,
  0.14709982820715783),
 (61,
  StandardScaler(),
  'uniform',
  'ball_tree',
  12,
  0.4793929703153893,
  0.6264927985225471,
  0.14709982820715783),
 (61,
  StandardScaler(),
  'uniform',
  'kd_tree',
  12,
  0.4793929703153893,
  0.6264927985225471,
  0.14709982820715783),
 (61,
  RobustScaler(),
  'uniform',
  'auto',
  12,
  0.4804508448475793,
  0.6258343928469541,
  0.1453835479993748),
 (61,
  RobustScaler(),
  'uniform',
  'ball_tree',
  12,
  0.4804508448475793,
  0.6258343928469541,
  0.1453835479993748),
 (61,
  Ro

In [12]:
scoreDF = pd.DataFrame(scoreDict).T
scoreDF.columns = [
    "random_state",
    "scaler",
    "weight",
    "algorithm",
    "k",
    "train_score",
    "test_score",
    "diff",
]
scoreDF


,random_state,scaler,weight,algorithm,k,train_score,test_score,diff
1,0,StandardScaler(),uniform,auto,15,0.499288,0.50464,0.005352
2,0,StandardScaler(),uniform,auto,16,0.498496,0.500183,0.001688
3,0,StandardScaler(),uniform,auto,17,0.493364,0.50725,0.013886
4,0,StandardScaler(),uniform,auto,18,0.49163,0.512868,0.021238
5,0,StandardScaler(),uniform,auto,19,0.488105,0.512092,0.023988
...,...,...,...,...,...,...,...,...
24722,99,QuantileTransformer(),distance,brute,46,1.0,0.530273,0.469727
24723,99,QuantileTransformer(),distance,brute,47,1.0,0.534123,0.465877
24724,99,QuantileTransformer(),distance,brute,48,1.0,0.536999,0.463001
24725,99,QuantileTransformer(),distance,brute,49,1.0,0.536611,0.463389


In [13]:
bestDF_point_1 = scoreDF[scoreDF["diff"] <= 0.1].sort_values(
    by="test_score", ascending=False
)[:1]
bestDF_point_2 = scoreDF[scoreDF["diff"] <= 0.2].sort_values(
    by="test_score", ascending=False
)[:1]
bestDF = pd.concat([bestDF_point_1, bestDF_point_2])


In [14]:
bestDF


,random_state,scaler,weight,algorithm,k,train_score,test_score,diff
20957,70,QuantileTransformer(),uniform,brute,7,0.510851,0.599955,0.089104
18973,61,RobustScaler(),uniform,kd_tree,10,0.49002,0.628885,0.138865


In [15]:
def model_fit(random_state, scaler, weight, algorithm, k):
    x_train, x_test, y_train, y_test = train_test_split(
        featureDF, targetDF, test_size=0.2, random_state=random_state
    )
    scaler.fit(x_train)
    scaled_x_train = scaler.transform(x_train)
    scaled_x_test = scaler.transform(x_test)
    model = KNeighborsRegressor(n_neighbors=k, weights=weight, algorithm=algorithm)
    model.fit(scaled_x_train, y_train)
    train_score = model.score(scaled_x_train, y_train)
    test_score = model.score(scaled_x_test, y_test)
    diff = abs(test_score - train_score)
    y_pre = model.predict(scaled_x_test)
    mae = mean_absolute_error(y_test, y_pre)
    mse = mean_squared_error(y_test, y_pre)
    rmse = mean_squared_error(y_test, y_pre, squared=False)

    print("train score :", train_score)
    print("test score  :", test_score)
    print("difference  :", diff)
    print("MAE         :", mae)
    print("MSE         :", mse)
    print("RMSE        :", rmse)


In [16]:
for i in range(2):
    print(f"결과 {i+1} : 오차 {bestDF.values[i,7]:.4f}")
    model_fit(*bestDF.values[i][:5])
    print()


결과 1 : 오차 0.0891
train score : 0.5108508113642116
test score  : 0.5999552504842806
difference  : 0.08910443912006893
MAE         : 40.69767441860465
MSE         : 2449.978168011391
RMSE        : 49.49725414617856

결과 2 : 오차 0.1389
train score : 0.490020473393589
test score  : 0.6288849806298598
difference  : 0.13886450723627075
MAE         : 37.05581395348837
MSE         : 2316.710697674418
RMSE        : 48.132220992536986



c:\Users\KDP-25\.conda\envs\MachineLearning39\lib\site-packages\sklearn\preprocessing\_data.py:2667: UserWarning: n_quantiles (1000) is greater than the total number of samples (344). n_quantiles is set to n_samples.
  warnings.warn(


In [24]:
diabetesDF.describe()


,AGE,SEX,BMI,BP,tc,ldl,hdl,tch,ltg,glu,Y
count,430.000000,430.000000,430.000000,430.000000,430.000000,430.000000,430.000000,430.000000,430.000000,430.000000,430.000000
mean,48.504651,1.469767,26.289302,94.674372,188.025581,114.847209,49.477907,4.043395,4.634329,91.102326,151.393023
std,13.203578,0.499667,4.281714,13.767554,33.803033,29.473002,11.969857,1.218012,0.519470,11.491712,76.278513
min,19.000000,1.000000,18.000000,62.000000,97.000000,41.600000,22.000000,2.000000,3.258100,58.000000,25.000000
25%,38.250000,1.000000,23.200000,85.000000,164.000000,96.050000,41.000000,3.000000,4.266200,83.000000,87.250000
50%,50.000000,1.000000,25.700000,93.000000,186.000000,113.000000,48.000000,4.000000,4.605200,91.000000,140.000000
75%,59.000000,2.000000,29.200000,104.917500,208.000000,133.450000,57.000000,5.000000,4.988700,98.000000,207.500000
max,79.000000,2.000000,39.100000,133.000000,286.000000,199.400000,88.000000,7.900000,6.107000,124.000000,341.000000


In [27]:
diabetesDF["ltg"]


0      4.8598
1      3.8918
2      4.6728
3      4.8903
4      4.2905
        ...  
425    3.9703
426    4.9836
427    4.4427
428    4.1271
429    5.1299
Name: ltg, Length: 430, dtype: float64

In [29]:
bins_tc = [0, 199, 239, 300]
tc_cut = pd.cut(diabetesDF["tc"], bins=bins_tc, labels=False)
bins_ldl = [0, 99.9, 129.9, 159.9, 189.9, 200]
ldl_cut = pd.cut(diabetesDF["ldl"], bins=bins_ldl, labels=False)
bins_hdl = [0, 59.9, 90]
hdl_cut = pd.cut(diabetesDF["hdl"], bins=bins_hdl, labels=False)
bins_ltg = [0, 4.6, 4.9, 5.2, 7]
ltg_cut = pd.cut(diabetesDF["ltg"], bins=bins_ltg, labels=False)


In [32]:
np.corrcoef(diabetesDF["Y"], tc_cut)


array([[1.        , 0.19595355],
       [0.19595355, 1.        ]])

In [33]:
np.corrcoef(diabetesDF["Y"], ldl_cut)


array([[1.        , 0.15126583],
       [0.15126583, 1.        ]])

In [34]:
np.corrcoef(diabetesDF["Y"], hdl_cut)


array([[ 1.        , -0.29313946],
       [-0.29313946,  1.        ]])

In [35]:
np.corrcoef(diabetesDF["Y"], ltg_cut)


array([[1.        , 0.55109381],
       [0.55109381, 1.        ]])

In [36]:
test = tc_cut + ldl_cut + hdl_cut + ltg_cut


In [38]:
np.corrcoef(diabetesDF["Y"], test)


array([[1.        , 0.35606411],
       [0.35606411, 1.        ]])